In [1]:
# !pip install tensorflow==2.7.0

In [2]:
# !pip list

In [3]:
import datetime as dt
import re
import subprocess
# import tensorflow as tf
import time

from google.cloud import bigquery
from pyspark import SparkConf, SparkContext, SparkFiles
from pyspark.sql import SQLContext
from pyspark.streaming import StreamingContext

In [4]:
# global variables
bucket = "6893_project_data"
saved_models_dir = "gs://{}/saved_models/".format(bucket)
output_data_dir = "gs://{}/output/twitter_stream".format(bucket)

In [5]:
# parameter
IP = "localhost"  # ip port
PORT = 9001  # port

STREAM_TIME = 60  # time that the streaming process runs
INTERVAL = 5

In [6]:
# Helper functions
def save_to_storage(rdd, output_directory, column_name, mode):
    """
    Save each RDD in this DStream to google storage
    Args:
        rdd: input rdd
        output_directory: output directory in google storage
        column_name: columns name of dataframe
        mode: mode = "overwrite", overwrite the file
              mode = "append", append data to the end of file
    """
    if not rdd.isEmpty():
        data = rdd.toDF()
        print(data)
        data.write.save(output_directory, format = "json", mode = mode)

In [7]:
def clean_text(text):
    text = re.sub(r"https?:\/\/\S*\s?", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = re.sub(r"\n", " ", text)
    return text

In [8]:
# Spark settings
conf = SparkConf()
conf.setMaster("local[2]")
conf.setAppName("TwitterStreamApp")

In [9]:
# Create spark context with the above configuration
sc = SparkContext(conf = conf)
sc.setLogLevel("ERROR")

In [10]:
# Create sql context, used for saving rdd
sql_context = SQLContext(sc)

In [11]:
# create the Streaming Context from the above spark context with batch interval size 5 seconds
ssc = StreamingContext(sc, INTERVAL)
# setting a checkpoint to allow RDD recovery
ssc.checkpoint("~/checkpoint_TwitterApp")

In [12]:
# read data from port 9001
data_stream = ssc.socketTextStream(IP, PORT)
data_stream.pprint()

In [13]:
data_stream = data_stream.map(lambda t: clean_text(t))
data_stream.pprint()

In [14]:
data_stream.foreachRDD(lambda rdd: save_to_storage(rdd, output_data_dir, ["text"], mode = "overwrite"))

In [15]:
# !pip install tensorflow-text

In [16]:
# import tensorflow_text as text

In [17]:
# model = tf.keras.models.load_model(saved_models_dir + "twitter_bert", compile = False)

In [18]:
# def predict_text(data):
#     sent_pred = data.map(lambda t: model.predict([t]))
#     return sent_pred

In [19]:
# text_sent = predict_text(data_stream)
# text_sent.pprint()

In [ ]:
# start streaming process, wait for 600s and then stop.
ssc.start()
time.sleep(STREAM_TIME)
ssc.stop(stopSparkContext = False, stopGraceFully = True)

-------------------------------------------
Time: 2022-12-21 02:12:00
-------------------------------------------

-------------------------------------------
Time: 2022-12-21 02:12:00
-------------------------------------------

-------------------------------------------
Time: 2022-12-21 02:12:05
-------------------------------------------

-------------------------------------------
Time: 2022-12-21 02:12:05
-------------------------------------------

-------------------------------------------
Time: 2022-12-21 02:12:10
-------------------------------------------

-------------------------------------------
Time: 2022-12-21 02:12:10
-------------------------------------------

-------------------------------------------
Time: 2022-12-21 02:12:15
-------------------------------------------

-------------------------------------------
Time: 2022-12-21 02:12:15
-------------------------------------------

-------------------------------------------
Time: 2022-12-21 02:12:20
----------